### 一. 使用缓存的目的是什么？
#### 1. 高性能（缓存的数据要求一段时间内不发生变化）
 * 不使用用缓存：一个用户访user1问查询接口， 接口直接查询数据库（加入耗时800毫秒）；此后user2再次查询同一条数据， 则又会耗时800毫秒；假设数据在10分钟内没有变化，10分钟内有1000个用户查询的话， 就会耗时1000个800毫秒；
 * 使用缓存后： 第一次查询时，缓存中没有，则需要查询数据库，耗时800毫秒；但由于数据在十分钟内没有变化，因此此后的999次查询都可以从内存中直接查询， 每次耗时10ms，共耗时999个10毫秒；
 * 或者某些复杂查询可能很耗时，缓存后可以直接查出
 
#### 2. 高并发  
* 数据库如果每秒4000个请求，则数据库可能会宕机；   
 如果对每个query，先在缓存中查询，查不到再使用数据库查询；这种方法会让4000个请求分散成3000次缓存查询+1000次数据库查询
* 为什么缓存比数据库更能支持高并发？   
 因为缓存是内存查询，支持高并发查询；而数据库需要磁盘，一般建议每秒查询次数一般不超过2000次

### 三. 为什么redis这种单线程模型比多线程的memcache有更高的性能？
#### 1. redis与memcached的区别？  
   redis支持的数据类型比memcached更多，更能支持复杂应用     
   memcached没有原声的分布式集群， 需要客户端进行分片后再写；redis原生支持cluster模式 
#### 2. **redis的线程模型？**   
1. redis基于io多路复用的网络模型进行IO, 使用一个叫做"文件事件处理器"的东西处理接受连接,可读,可写,关闭事件:  
   即可理解为serversocketChannel注册了4种状态的selectionkey, 用selector轮询检查
    * key.isReadable()
    * key.isWriteable()
    * key.isAccept()
    * key.isClose()
2. 上面注册的4种状态的socket会被放入对应的四种队列中, selector选择一种状态的socket后, 文件事件处理器的4个不同部分会做出处理, 由于文件事件处理器是单线程的, 即处理完1个socket才能处理下一个socket, 所以redis是单线程的  
    
### 四. redis有哪些数据类型, 有哪些应用场景
(面向初级开发者)  
* string
* list
* set
* zset : sortedset, 相当于map结构, 但是对每个value赋予一个score权重, 同一个key, 不同的value不会被覆盖, 而是按照score排序存储

### 五. 如何实现redis的过期策略
#### 1. redis中的数据会丢失?  
  redis是一个缓存, 而不是一个数据库, 既然是缓存, 当数据超过内存限制, 自然会丢弃一些过期数据
  
#### 2. 对某个key设置过期时间后, 数据在这个时间后仍然存在?  
    1. 这涉及到redis的一个内部逻辑, 如何删除那些设置了过期时间的key?      
    采用2种删除方式, 定期删除+惰性删除  
        * 定期删除: redis每隔100ms, 从所有设置超时时间的key中抽样一部分进行检查, 删除过期的kv对.(抽样检查的目的是, redis中可能存在大量设置过期时间的key, 如果全部检查会导致cpu负载全部花在检查上了)
        * 惰性删除: 当查询1个key时, 如果redis发现设置了过期时间且已经过期, 则会删除这个key并返回空
        
    2. 如上删除策略, 如果一些key既没有被定期删除采样到, 也没有在一段时间内继续访问, 那该如何删除呢?    
    这部分数据会在内存淘汰机制中被删除, 内存淘汰机制主要有以下几种(一般都是使用allkeys-lru机制)
        * allkeys-lru : 对所有key进行LRU
        * volatile-lru : 只对设置了过期时间的key进行lru, 淘汰最近最久未被使用的
        * noevication : 内存慢了以后在插入数据直接报错
        * allkeys-rendom : 选择一个key随机删除   
    
    以上需要在 redis.conf中配置最大内存使用大小, 然后配置内存淘汰策略   
    3. 如何实现一个LRU缓存(双向链表+HashTable)

### 六. 生产环境下, redis如何实现高并发高可用机制?
#### 1. redis如何承载高并发? 实现每秒10万+的QPS (**master-slave读写分离结构**)?  
    1. 单节点的redis, qps一般在1万左右, 想承载10万qps.必须上redis的读写分离模式  
    2. qps是每秒处理的查询数量, 就表示这是一个读多写少的场景, 也正是缓存应用的场景; 如果是写多读少, 就无法使用分布式缓存了, 需要使用一些列复杂的异步消息队列实现. 
    3. redis的读写分离是这样的, 使用单个master节点负责写, master会向slave节点异步同步数据,其余slave节点负责读; 这种架构的好处在于, 对于处理读请求的slave节点可以水平扩容来满足qps; 
<img src="img/redis1.png" width="60%">

**redis的各种架构演练**    
`redis replication` -> `redis 主从架构` -> `redis 读写分离架构` -> `水平扩容支持高并发`  
**总结: redis如何实现高并发**  
1. 可以使用redis的主从架构: 一主多从. 主负责写请求, 从负责并发的读请求; 这种架构可以实现十万的qps; 通过部署哨兵, 可以实现故障的自动转移  
2. 如果要在实现高并发的同时还要实现大数据量, 就要上redis的集群模式(后面讲解).  
   因为主从架构下的redis, 每个节地那都是全量的数据, 如果主有10g内存, 那么整个缓存只能缓存10g的数据

#### 2. redis replication:  
    1. redis master节点采用异步方式向slave节点复制数据; 不过从redis2.0开始, slave节点也会定期向master确认自己的数据复制量  
    2. 1个redis master节点, 可以配置多个slave节点
    3. slave节点也可以连接其它slave节点
    4. slave在复制master的数据时, 不会block自己的查询请求, 只不过此事的查询请求是在老的数据集上进行的;   
      但是slave复制数据完成之后, 需要先删除旧数据集, 加载新数据集, 此时会block查询请求  
    5. slave 节点主要用来横向扩容, 做读写分离, 提高qps
    6. master节点为什么要开启持久化, 它对安全性的意义是什么?   
      因为如果master不开启持久化, 当master宕机后, 再次重启会发现当前没有数据, 此时mster向slave同步的数据也是空的, 这会导致原本存在slave上的数据也被100%丢失

#### 3. redis数据丢失的两种情况  
    1. 异步数据复制导致数据丢失
      因为master数据时一步复制的, 如果slave复制的延迟很高, 而此时master宕机, 就会导致master的数据没来得急复制到slave而丢失  
      
    2. 脑裂, 导致多个master存在  
      脑裂是因为网络分区而产生的, 网络分区后, sentinal和slave在一个分区, master自己在一个分区, 此时sentinal会把另外一个slave提升成mstaer, 而原来还存在1个master可以负责读, 导致2个master数据不一致.
    
    3. 参数设置减轻`数据异步复制`和`脑裂`导致的数据丢失问题  
       有2个参数可供修改:     
       min-slaves-to-write : 从节点小于多少个，主节点就不写了
       min-slaves-max-lag  : 从节点大于多少秒延迟，就不写了

#### 4. (Sentinal)底层原理 (23-9)？？

#### 5. redis持久化的意义
持久化的意义在于故障恢复. 如果不进行持久化, 一旦redis集群宕机, 重启后导致缓存内没有任何数据, 所有请求直接到后端mysql, 而mysql的并发支撑很小, 导致mysql马上宕机, 系统瘫痪  

#### 6. redis的2种持久化机制
* RDB : 每隔一段时间, 产生一个内存中的数据快照
* AOF(append only file) : 记录每条"写命令"到os cache中, 每个1秒触发fsync()将os cache中的数据写入到磁盘的文件, 当aof文件膨胀到达一定大小时, 会触发rewrite机制, 基于当前内存中的数据, 产生一个新的小一点的aof文件


### 七. redis-cluster模式  
突破单机瓶颈, 让redis支持1T的数据量
#### 1. 思路: 横向扩展master节点  
* 还是基于1master多slave的架构, 但是部署多组1maser多slave, 通过让数据部署到多个master实现更大数据量的支撑
* redis-cluster模式 
    * 多master+读写分离+高可用
    * redis-cluster vs (replication(1主多从)+sentinal): 数据量小选择后者, 大选择前者
    
#### 2. 如何将数据分布到多master
[数据分布算法](../Distributed%20System/4-分布式缓存下的数据分布算法.ipynb): 经历了hash -> 一致性hash -> Redis的hash slot

#### 3. redis-cluster模式下的元数据维护(gossip协议)
集群模式下有多个master节点, 这些master节点要有相同的元数据进行管理. 元数据用来描述: 
* master -> slot之间的映射
* master -> slave之间的映射

gossip协议是指, 元数据在每个master节点上都存在, 如果某个节点需要修改元数据, 该节点将修改后的元数据同步给其它节点. 
与gossip协议对应的是`集中式`元数据管理, 一般使用zookeeper来做集中式元数据管理.
* gossip的优势在于, 元数据的更新会分散到每个master节点, 相比集中式管理下的压力更小; 
* 缺点是元数据的同步存在延迟, 可能会导致一段时间的集群不可用  
```
比如刚刚做完reshard后, 再做其他操作, 汇报configuration error, 表示元数据还未达成一致
```


gossip协议有几种消息:
* meet
* ping
* pong
* ..


### 八. [重要]  缓存雪崩和缓存穿透问题

#### 1.  缓存雪崩的解决方法
缓存雪崩的意思是: 一般通常使用架构为`接口 -> 查缓存 -> 未命中的查数据库`; 由于缓存宕机, 所有请求达到数据库, 导致数据库崩溃, 系统瘫痪

如何解决缓存雪崩?  
* 事前 : redis要采用高可用架构: redis-cluster模式或
* 事中 : 
    1. 不要只把数据缓存到redis, 可以把数据再缓存程序的内存缓存中, 比如使用ehcache;   
    2. 其次加入降级框架: 限制只有1000个请求可以走数据库, 其它请求返回错误码. 这样顶多是用户点击多次之后才出现结果, 不至于整个请求达到数据库后使服务瘫痪
* 事后 : redis开启持久化机制, 便于故障恢复


#### 2. 缓存穿透的解决办法
1. 什么是缓存穿透  
   5000个请求中有4000个是黑客攻击, 缓存和数据库中都没有, 这会导致每秒都有4000个请求达到数据库中, 是数据库无法处理正常查询;  
2. 如何解决缓存穿透  
   解决这个问题的办法是, 当数据库查询返回空时, 把空设置为一个查询结果值(eg:UNKNOWN), 并将<key,UNKONWN>放入缓存中, 下次再请求这个空值时, 可以直接返回结果  

### 九. Cache Aside Pattern(缓存+数据库读写)的一些问题?

#### 1. 说说缓存+数据库读写架构下, 读流程和写流程?
* 读: 先读缓存, 缓存未命中则从数据库中读, 并将结果写入到缓存中, 最后返回响应
* 更新: 先删除缓存, 再更新数据库

#### 2. 为什么更新数据时, 只是从缓存中删除数据, 而不是直接更新缓存中的数据?
主要有3个原因:
1. **复杂计算**: 很多时候, 缓存中的数据并不是简简单单的数据库中指的拷贝, 而是多个字段计算的结果. 对某个字段值得更改无法获得最终缓存结果值得更改  
2. **冷数据**: 全量的数据库数据中, 大概只有20%的数据是高频数据, 需要被缓存. 某个数据, 也许1分钟内被频繁修改100次, 但1分钟内却只被查询1次, 这种数据没有必要做缓存; 其次, 只删除缓存不更新缓存的做法也是一种`lazy value`的做法: 数据只有被用到的时候才被计算  
3. **避免缓存和数据库中数据不一致**:  
   如果同时更新缓存和数据库,有可能缓存更新成功, 数据库更新失败, 则导致数据不一致; 如果只删除缓存, 无论数据库更新成功还是失败, 数据都只依照数据库为基准;

#### 3. 高并发场景下, 缓存+数据库双写不一致问题如何解决?
1. 最初级的缓存不一致及解决方案  
   更新数据时, 先删除缓存再更新数据库
2. 并发场景下的缓存双写不一致  
   上面说到先删除缓存,再更新数据库.在并发量高的多线程场景中, 仍可能出现缓存不一致问题. 如下按照时间顺序描述发生的事情:  
    1. 线程A: 完成了缓存的删除, 还未更新数据库, 此时时间片轮转给了线程B; 
    2. 线程B从数据库中读取了一个老值并放回了缓存中, 此后时间片轮转给了线程A;  
    3. 线程A去更新数据库中的值为新值; 造成了数据库和缓存的数据不一致  
    
  如何解决高并发下的数据库双写不一致?   
     1. 在内存中设置500个内存队列, 将每个key计算hash后取余, 找到key所述的内存队列. 即每个key都能路由到一个内存队列中  
     2. 将对同一个key的操作缓存在队列中, 按顺序执行, 就避免了双写不一致  
     (此处有一个优化点, 读请求不必入队列 ,读请求先检查自己所属的队列上是否有写请求了, 若有可以轮训等待一会儿, 超时时间过后若发现队列中还有写操作未完成, 则只查数据库后直接返回)  
     
 这种解决办法适用的场景?  
     1. 这种解决办法适用于写频率不是很高的场景. 因为写频率如果太高, 内存队列中的写积压的很多, 比如积压了100个. 此时来了一个读请求, 需要等这100个写请求完成后才能正常读(如果发生这种情况, 只能横向扩展部署服务的机器, 使每个服务处理的数据减少, 从而让内存队列中积压的操作减少)  
     2. 这种方案, 要在压测后, 找到1s内内存队列上有多少个写请求阻塞了多少读请求  